# Lab2

---

## Task

Реализовать метод решения СЛАУ, на выбор: LU-разложение или метод квадратного корня. Для матриц A, L, U вычислить числа обусловленности (см. задание 1).

Протестировать на разных матрицах: хорошо обусловленных, (очень) плохо обусловленных.

Для нескольких плохо обусловленных матриц (например, для матриц Гильберта разного, больше 15, порядка) реализовать метод регуляризации:
- параметр α варьировать в пределах от 10<sup>-12</sup> до 10<sup>-1</sup>
- для каждого конкретного значения α найти числа обусловленности (матриц A + αE) и норму погрешности получившегося решения
- понять, какое значениe α = α в каждом конкретном случае (= для каждой конкретной матрицы) кажется наилучшим

Наилучшее α можно
- находить из предположений, что точным решением является вектор x_0 = (1, 1, ... , 1)<sup>T</sup>
- находить из предположений, что точным решением является случайный вектор x_0
Проверить результат на (другом) случайном векторе x_0.

## Solution

--- 

LU decomposition is used.

In [ ]:
from scipy import linalg as la

def lu_solve(A, b):
    lu, pivot = la.lu_factor(A)
    sol = la.lu_solve((lu, pivot), b)
    return sol

## Experimental research

---

In [ ]:
from utils.cond_nums import *
from utils.matrices import *

In [ ]:
def print_cond_nums_for_A_L_U(A):
    _, L, U = la.lu(A)
    for name, matr in (("A", A), ("L", L), ("U", U)):
        spec_cn, bulk_cn, ang_cn = compute_matr_condition_nums(matr)
        print(f"Matrix {name}:")
        print(f"    Spectral condition number : {spec_cn}")
        print(f"    Bulk condition number     : {bulk_cn}")
        print(f"    Angular condition number  : {ang_cn}")

### Tests on different matrices:
- randomly generated matrix
- randomly generated sparse matrix
- tridiagonal diagonally dominant matrix
- hilbert matrices

In [ ]:
rank = 50

In [ ]:
A = create_random_matrix(rank)
print_cond_nums_for_A_L_U(A)

In [ ]:
A = create_random_sparse_matrix(rank)
print_cond_nums_for_A_L_U(A)

In [ ]:
A = create_tridiagonal_matrix(rank)
print_cond_nums_for_A_L_U(A)

### Hilbert matrices

First we evaluate the alpha using regularization method

In [ ]:
import numpy as np

def alpha_evaluation(matr, alpha_range):
    """
    Regularization method
    """
    n, m = matr.shape
    id_vect = np.ones(m)
    b = matr.dot(id_vect)
    
    differences = []
    stability = []

    for alpha in alpha_range:
        varied_matr = matr + np.eye(m) * 10 ** alpha
        
        sol_lu = lu_solve(varied_matr, b)

        difference = la.norm(id_vect - sol_lu)
        differences.append(difference)

        cond_number = bulk_condition_nums(varied_matr)
        stability.append(cond_number)

    return differences, stability

alpha_range = range(-12, -1)
rank = 30
hilbert_matrix = create_hilbert_matrix(rank)
differences, stability = alpha_evaluation(hilbert_matrix, alpha_range)

In [ ]:
import matplotlib.pyplot as plt

plt.title("Alpha evaluation")
plt.plot(alpha_range, differences)
plt.xlabel("Alpha")
plt.ylabel("Error norm")
plt.show()

plt.title("Solution stability")
plt.plot(alpha_range, stability)
plt.yscale("log", base=10)
plt.xlabel("Alpha")
plt.ylabel("Conditional number")
plt.show()

Then we test alpha value on other x_0

In [ ]:
alpha = 10 ** (-6)

def experiment(matr, alpha):
    n, m = matr.shape
    x_0 = np.random.rand(m)
    b = matr.dot(x_0)

    varied_matr = matr + np.eye(m) * 10 ** alpha
    sol_lu = lu_solve(varied_matr, b)

    return la.norm(x_0 - sol_lu)

print(experiment(hilbert_matrix, alpha))